In [1]:
import gradio as gr

from dotenv import load_dotenv
from pathlib import Path
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [2]:
load_dotenv()

documents = []

folders = Path(".").glob("knowledge-base/*")
for folder in folders:
    if folder.is_dir():
        loader = DirectoryLoader(folder, loader_cls = TextLoader)
        docs = loader.load()
        documents += docs

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Chunks created: {len(chunks)}")

Created a chunk of size 1088, which is longer than the specified 1000


Chunks created: 123


In [3]:
embeddings = OpenAIEmbeddings()
db_name = "vector_db"
db_path = Path(db_name)

if db_path.exists():
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(chunks, persist_directory=db_name, embedding=embeddings)
print(f"Vectorstore created with {vectorstore._collection.count()} documents.")

Vectorstore created with 123 documents.


In [ ]:
!pip install -U langchain_ollama
!pip install -U langchain_core.messages


In [ ]:
import langchain_core.messages
dir (langchain_core.messages)

In [ ]:
from langchain_ollama import OllamaEmbeddings

In [ ]:
ollama_embeddings = OllamaEmbeddings(model="ollama3")

In [5]:
llm = ChatOpenAI(model_name="gpt-5")
retriever = vectorstore.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

/var/folders/ny/bjrptr9d4f9c3mmxj10zzcl40000gp/T/ipykernel_49135/859997501.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [6]:
resp = chain.invoke({"question": "Who is Lancaster?"})
print(resp["answer"])

Avery Lancaster is the Co-Founder and CEO of Insurellm, based in San Francisco. She co-founded the company in 2015 and has led its growth into a leading insurance tech provider, known for innovative leadership and risk management. Previously, she was a Senior Product Manager at Innovate Insurance Solutions (2013–2015), and she champions diversity, flexible work, and community financial literacy initiatives.


In [7]:
def chat(message, history):
    resp = chain.invoke({"question": message})
    return resp["answer"]

ui = gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
